In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras import layers
from keras.callbacks import EarlyStopping
import tensorflow as tf
import sklearn, os

from mpl_toolkits import mplot3d

/home/yloukachev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset_names = {'pCO2': 'pCO2_2D_mon_CESM001_1x1_198201-201701.nc',
                 'XCO2': 'XCO2_1D_mon_CESM001_native_198201-201701.nc',
                 'SST': 'SST_2D_mon_CESM001_1x1_198201-201701.nc',
                 'SSS': 'SSS_2D_mon_CESM001_1x1_198201-201701.nc',
                 'MLD': 'MLD_2D_mon_CESM001_1x1_198201-201701.nc',
                 'Chl': 'Chl_2D_mon_CESM001_1x1_198201-201701.nc'}

ds = {}
for dataset in dataset_names.keys():
    filename = os.path.join(dataset_names[dataset])
    ds[dataset] = xr.open_dataset(filename)

In [4]:
merged_dataset = xr.merge([ds[name][name] for name in ds.keys()])
df = merged_dataset.to_dataframe().reset_index()

df.dropna(subset=['xlon', 'ylat','pCO2', 'XCO2', 'SST', 'SSS', 'MLD', 'Chl'], inplace=True)
shift_param = int(df.shape[0]/df['time'].nunique())

df['A'], df['B'], df['C'] = np.sin(df['ylat']), np.sin(df['xlon'])*np.cos(df['ylat']), -np.cos(df['xlon'])*np.cos(df['ylat'])
df['T0'] = np.cos(df.time.dt.dayofyear * 2 * np.pi / 365)
df['T1'] = np.sin(df.time.dt.dayofyear * 2 * np.pi / 365)

df.drop(columns=['time', 'TLONG', 'TLAT'], inplace=True)

df['pCO2'] = df.pop('pCO2')
df.head()

,xlon,ylat,XCO2,SST,SSS,MLD,Chl,A,B,C,T0,T1,pCO2
20,0.5,-69.5,340.848541,-1.552322,33.641834,12.729663,1.128493,-0.375524,0.444338,-0.813355,0.962309,0.271958,256.390077
21,0.5,-68.5,340.848541,-1.626047,33.667816,19.951666,0.900315,0.576990,0.391571,-0.716767,0.962309,0.271958,262.338616
22,0.5,-67.5,340.848541,-1.607591,33.572289,19.510101,0.635458,0.999021,-0.021204,0.038813,0.962309,0.271958,261.956626
23,0.5,-66.5,340.848541,-1.496069,33.424641,20.714338,0.420227,0.502557,-0.414484,0.758709,0.962309,0.271958,262.739347
24,0.5,-65.5,340.848541,-1.207762,33.317802,20.579535,0.380468,-0.455956,-0.426690,0.781051,0.962309,0.271958,266.288585


## Baseline Feedforward Neural Network

In [ ]:
df.drop(columns=['ylat', 'xlon'], inplace=True)

values = df.values
n_train_months = shift_param * 12 * 30 # training on 30 years and evaluating on five years

train = values[:n_train_months, :]
test = values[n_train_months:, :]

train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1024, activation=tf.nn.relu, kernel_initializer='glorot_normal'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_initializer='glorot_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, kernel_initializer='normal')])

model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mse'])

In [ ]:
history = model.fit(train_X, train_y, epochs=100, batch_size=1024, validation_data=(test_X, test_y))

In [ ]:
mse = history.history['mean_squared_error']
val_mse = history.history['val_mean_squared_error']

epochs = range(1, len(mse) + 1)

plt.figure()
plt.plot(epochs, mse, 'r', label='Training MSE')
plt.plot(epochs, val_mse, 'b', label='Validation MSE')
plt.title('Training and validation MSE')
plt.legend()

plt.show()

In [ ]:
model.save_weights('baseline_DNN.h5')

## SOM Clusters as Input to FFN

In [4]:
merged_dataset = xr.merge([ds[name][name] for name in ds.keys()])
df = merged_dataset.to_dataframe().reset_index()

df.dropna(subset=['xlon', 'ylat','pCO2', 'XCO2', 'SST', 'SSS', 'MLD', 'Chl'], inplace=True)
shift_param = int(df.shape[0]/df['time'].nunique())

df['A'], df['B'], df['C'] = np.sin(df['ylat']), np.sin(df['xlon'])*np.cos(df['ylat']), -np.cos(df['xlon'])*np.cos(df['ylat'])
df['T0'] = np.cos(df.time.dt.dayofyear * 2 * np.pi / 365)
df['T1'] = np.sin(df.time.dt.dayofyear * 2 * np.pi / 365)

df.drop(columns=['time', 'TLONG', 'TLAT'], inplace=True)

df['pCO2'] = df.pop('pCO2')

In [5]:
SOM_df = pd.read_csv('means_SOM.csv', index_col=0)
SOM_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

SOM_df = pd.merge(df, SOM_df[['ylat', 'xlon', 'k=5', 'k=10', 'k=15']], how='inner', left_on=['ylat', 'xlon'], right_on=['ylat', 'xlon'])

SOM_df['A'], SOM_df['B'], SOM_df['C'] = np.sin(SOM_df['ylat']), np.sin(SOM_df['xlon'])*np.cos(SOM_df['ylat']), -np.cos(SOM_df['xlon'])*np.cos(SOM_df['ylat'])
SOM_df.drop(columns=['ylat', 'xlon'], inplace=True)

SOM_df['pCO2'] = SOM_df.pop('pCO2')
SOM_df.head()

,XCO2,SST,SSS,MLD,Chl,A,B,C,T0,T1,k=5,k=10,k=15,pCO2
0,340.848541,-1.552322,33.641834,12.729663,1.128493,-0.375524,0.444338,-0.813355,0.962309,0.271958,3,0,7,256.390077
1,340.962250,-1.643371,33.400482,22.005390,0.064728,-0.375524,0.444338,-0.813355,0.702527,0.711657,3,0,7,259.943180
2,341.075439,-1.800755,33.382126,38.298309,0.115517,-0.375524,0.444338,-0.813355,0.276097,0.961130,3,0,7,271.485986
3,341.193176,-1.816007,33.603302,65.371696,0.062877,-0.375524,0.444338,-0.813355,-0.251190,0.967938,3,0,7,286.451083
4,341.310730,-1.814708,33.780853,83.675964,0.036665,-0.375524,0.444338,-0.813355,-0.696376,0.717677,3,0,7,300.785596


In [6]:
def train_test_split_SOM(df, num_clusters, test_split=0.2):
    if num_clusters == 5:
        df_cluster = df.drop(columns=['k=10', 'k=15'])
    elif num_clusters == 10:
        df_cluster = df.drop(columns=['k=5', 'k=15'])
    else:
        df_cluster = df.drop(columns=['k=5', 'k=10'])

    cluster_type = df_cluster.columns[-2]
    # creating a unique dataframe for each cluster
    df_dict = {}
    for k in df_cluster[cluster_type].unique():
        df_dict['df{0}'.format(k)] = df_cluster[df_cluster[cluster_type] == k].drop(columns=cluster_type)

    # creating a unique feature and target matrix for each cluster
    X_dict, y_dict = {}, {}
    for key, value in df_dict.items():
        X_dict['X{0}'.format(key[-1])] = value.values[:, :-1]
        y_dict['y{0}'.format(key[-1])] = value.values[:, -1:]

    # creating a unique train/test split for each feature/target mapping
    train_X_dict, test_X_dict, train_y_dict, test_y_dict = {}, {}, {}, {}
    for i in range(0, num_clusters):
        train_X_dict[i], test_X_dict[i], train_y_dict[i], test_y_dict[i] = train_test_split(X_dict['X{0}'.format(i)], y_dict['y{0}'.format(i)], test_size=test_split, shuffle=False)
        
    return train_X_dict, test_X_dict, train_y_dict, test_y_dict

In [7]:
clusters = 5

train_X, test_X, train_y, test_y = train_test_split_SOM(SOM_df, clusters)

In [8]:
def FFN_Model():
    model = Sequential()
    
    model.add(layers.Dense(1024, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(512, activation='relu', kernel_initializer='glorot_normal'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, kernel_initializer='glorot_normal'))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

In [ ]:
history_dict = {}

for k in range(clusters):
    history_dict['history_{}'.format(k)] = FFN_Model().fit([train_X[k]], [train_y[k]], validation_data=(test_X[k], test_y[k]), epochs=100)

In [ ]:
mse = history.history['mean_squared_error']
val_mse = history.history['val_mean_squared_error']

epochs = range(1, len(mse) + 1)

plt.figure()
plt.plot(epochs, mse, 'bo', label='Training MSE')
plt.plot(epochs, val_mse, 'b', label='Validation MSE')
plt.title('Training and validation MSE')
plt.legend()

plt.show()